# Transformer Model Testing in Google Colab

This notebook provides comprehensive testing functionality for your trained Transformer model.

## 1. Setup and Mount Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
import os
import sys
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

# Mount drive
drive.mount('/content/drive')

# Add path for imports
sys.path.append('/content/drive/MyDrive/')

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Import Required Modules

In [ ]:
# Import your modules
from encoder import TransformerEncoder
from decoder import TransformerDecoder, Transformer
from utils import (
    load_data, split_data, create_vocabulary, TransformerDataset,
    create_padding_mask, create_look_ahead_mask, calculate_bleu,
    indices_to_sentence, Vocabulary
)

print("All modules imported successfully!")

## 3. Load Trained Model

In [ ]:
# Load the best trained model
model_path = '/content/drive/MyDrive/models/best_model.pt'  # Adjust path as needed

print(f"Loading model from {model_path}")
checkpoint = torch.load(model_path, map_location=device)

# Extract configuration and vocabularies
src_vocab = checkpoint['src_vocab']
tgt_vocab = checkpoint['tgt_vocab']
config = checkpoint['args']

print(f"Model configuration:")
print(f"  Positional encoding: {config['pos_encoding_type']}")
print(f"  Model dimension: {config['d_model']}")
print(f"  Number of heads: {config['num_heads']}")
print(f"  Encoder layers: {config['num_encoder_layers']}")
print(f"  Decoder layers: {config['num_decoder_layers']}")
print(f"  Source vocab size: {len(src_vocab)}")
print(f"  Target vocab size: {len(tgt_vocab)}")
print(f"  Training epoch: {checkpoint['epoch']}")
print(f"  Validation loss: {checkpoint['val_loss']:.4f}")

## 4. Initialize Model

In [ ]:
# Create model with same configuration
model = Transformer(
    src_vocab_size=len(src_vocab),
    tgt_vocab_size=len(tgt_vocab),
    d_model=config['d_model'],
    num_heads=config['num_heads'],
    num_encoder_layers=config['num_encoder_layers'],
    num_decoder_layers=config['num_decoder_layers'],
    d_ff=config['d_ff'],
    max_seq_len=config['max_seq_len'],
    dropout=config['dropout'],
    pos_encoding_type=config['pos_encoding_type']
).to(device)

# Load trained weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Model loaded successfully!")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

## 5. Helper Functions

In [ ]:
def sentence_to_indices(sentence, vocab, max_length=None):
    """Convert sentence to indices"""
    words = sentence.lower().strip().split()
    indices = [vocab.get_idx(word) for word in words]

    if max_length is None:
        max_length = config['max_seq_len']

    if len(indices) > max_length - 2:  # -2 for SOS and EOS
        indices = indices[:max_length - 2]

    # Add SOS and EOS
    indices = [vocab.get_idx(vocab.SOS_TOKEN)] + indices + [vocab.get_idx(vocab.EOS_TOKEN)]

    # Pad if necessary
    while len(indices) < max_length:
        indices.append(vocab.get_idx(vocab.PAD_TOKEN))

    return indices

def translate_greedy(sentence, max_length=50):
    """Translate a sentence using greedy decoding"""
    model.eval()

    # Tokenize and convert to indices
    src_indices = sentence_to_indices(sentence, src_vocab)
    src = torch.tensor([src_indices]).to(device)

    # Create source mask
    src_mask = create_padding_mask(src, src_vocab.get_idx(src_vocab.PAD_TOKEN))

    # Encode source
    encoder_output = model.encoder(src, src_mask)

    # Start with SOS token
    tgt_indices = [tgt_vocab.get_idx(tgt_vocab.SOS_TOKEN)]

    with torch.no_grad():
        for _ in range(max_length):
            tgt = torch.tensor([tgt_indices]).to(device)
            tgt_mask = create_look_ahead_mask(tgt, tgt_vocab.get_idx(tgt_vocab.PAD_TOKEN))

            # Decode
            decoder_output = model.decoder(tgt, encoder_output, src_mask, tgt_mask)

            # Get next token
            next_token_logits = decoder_output[0, -1, :]
            next_token = torch.argmax(next_token_logits).item()

            tgt_indices.append(next_token)

            # Stop if EOS token
            if next_token == tgt_vocab.get_idx(tgt_vocab.EOS_TOKEN):
                break

    # Convert back to sentence
    return indices_to_sentence(tgt_indices, tgt_vocab)

def translate_beam_search(sentence, beam_size=4, max_length=50):
    """Translate a sentence using beam search"""
    model.eval()

    # Tokenize and convert to indices
    src_indices = sentence_to_indices(sentence, src_vocab)
    src = torch.tensor([src_indices]).to(device)

    # Create source mask
    src_mask = create_padding_mask(src, src_vocab.get_idx(src_vocab.PAD_TOKEN))

    # Encode source
    encoder_output = model.encoder(src, src_mask)

    # Initialize beams
    beams = [[tgt_vocab.get_idx(tgt_vocab.SOS_TOKEN)]]
    beam_scores = [0.0]

    with torch.no_grad():
        for step in range(max_length):
            candidates = []

            for i, beam in enumerate(beams):
                if beam[-1] == tgt_vocab.get_idx(tgt_vocab.EOS_TOKEN):
                    candidates.append((beam_scores[i], beam))
                    continue

                tgt = torch.tensor([beam]).to(device)
                tgt_mask = create_look_ahead_mask(tgt, tgt_vocab.get_idx(tgt_vocab.PAD_TOKEN))

                # Decode
                decoder_output = model.decoder(tgt, encoder_output, src_mask, tgt_mask)

                # Get probabilities for next token
                next_token_logits = decoder_output[0, -1, :]
                log_probs = F.log_softmax(next_token_logits, dim=-1)

                # Get top-k tokens
                top_k_probs, top_k_indices = torch.topk(log_probs, beam_size)

                for j in range(beam_size):
                    new_beam = beam + [top_k_indices[j].item()]
                    new_score = beam_scores[i] + top_k_probs[j].item()
                    candidates.append((new_score, new_beam))

            # Select top beams
            candidates.sort(key=lambda x: x[0], reverse=True)
            beams = [cand[1] for cand in candidates[:beam_size]]
            beam_scores = [cand[0] for cand in candidates[:beam_size]]

            # Check if all beams ended
            if all(beam[-1] == tgt_vocab.get_idx(tgt_vocab.EOS_TOKEN) for beam in beams):
                break

    # Return best beam
    best_beam = beams[0]
    return indices_to_sentence(best_beam, tgt_vocab)

print("Helper functions defined!")

## 6. Test Sample Sentences

In [ ]:
# Test on predefined sample sentences
sample_sentences = [
    "Hyvää huomenta",  # Good morning
    "Kiitos paljon",   # Thank you very much
    "Nähdään myöhemmin",  # See you later
    "Mikä on nimesi?",  # What is your name?
    "Pidän kahvista",  # I like coffee
    "Sää on kaunis tänään",  # The weather is beautiful today
    "Voitko auttaa minua?",  # Can you help me?
    "Olen opiskelija",  # I am a student
    "Missä on kauppa?",  # Where is the store?
    "Paljonko tämä maksaa?"  # How much does this cost?
]

print("Testing Sample Sentences:")
print("=" * 80)

for i, sentence in enumerate(sample_sentences):
    print(f"\n{i+1}. Finnish: {sentence}")

    try:
        greedy_trans = translate_greedy(sentence)
        beam_trans = translate_beam_search(sentence, beam_size=4)

        print(f"   Greedy: {greedy_trans}")
        print(f"   Beam:   {beam_trans}")

    except Exception as e:
        print(f"   Error: {e}")

## 7. Interactive Translation

In [ ]:
# Interactive translation - run this cell multiple times to test different sentences
finnish_sentence = input("Enter a Finnish sentence to translate: ")

if finnish_sentence.strip():
    print(f"\nInput: {finnish_sentence}")

    try:
        # Greedy translation
        greedy_translation = translate_greedy(finnish_sentence)
        print(f"Greedy Translation: {greedy_translation}")

        # Beam search translation
        beam_translation = translate_beam_search(finnish_sentence, beam_size=4)
        print(f"Beam Search Translation: {beam_translation}")

    except Exception as e:
        print(f"Error during translation: {e}")
else:
    print("Please enter a sentence to translate.")

## 8. Evaluate on Test Set

In [ ]:
# Load test data and evaluate BLEU score
print("Loading test data...")

# Load original data files
data_dir = Path('/content/drive/MyDrive/EUbookshop')
src_file = data_dir / 'EUbookshop.fi'
tgt_file = data_dir / 'EUbookshop.en'

if src_file.exists() and tgt_file.exists():
    # Load and split data (same as training)
    en_sentences, fi_sentences = load_data(str(tgt_file), str(src_file))
    _, _, test_data = split_data(en_sentences, fi_sentences, train_ratio=0.8, val_ratio=0.1)

    # Take a subset for evaluation (adjust size as needed)
    test_size = 50
    test_src = test_data[1][:test_size]  # Finnish sentences
    test_tgt = test_data[0][:test_size]  # English sentences

    predictions = []
    references = []

    print(f"Generating translations for {test_size} test sentences...")
    for i, src_sentence in enumerate(tqdm(test_src)):
        prediction = translate_greedy(src_sentence)
        predictions.append(prediction)
        references.append(test_tgt[i])

        if i < 5:  # Show first 5 examples
            print(f"\nExample {i+1}:")
            print(f"Source (FI): {src_sentence}")
            print(f"Reference (EN): {test_tgt[i]}")
            print(f"Prediction (EN): {prediction}")

    # Calculate BLEU score
    try:
        bleu_score = calculate_bleu(predictions, references)
        print(f"\n{'='*50}")
        print(f"BLEU Score: {bleu_score:.2f}")
        print(f"{'='*50}")
    except Exception as e:
        print(f"Error calculating BLEU score: {e}")

else:
    print("Test data files not found. Please ensure the data files are in the correct location.")

## 9. Model Information Summary

In [ ]:
# Display comprehensive model information
print("=" * 60)
print("MODEL SUMMARY")
print("=" * 60)

print(f"Model Architecture: Transformer")
print(f"Positional Encoding: {config['pos_encoding_type']}")
print(f"Model Dimension: {config['d_model']}")
print(f"Attention Heads: {config['num_heads']}")
print(f"Encoder Layers: {config['num_encoder_layers']}")
print(f"Decoder Layers: {config['num_decoder_layers']}")
print(f"Feed Forward Dimension: {config['d_ff']}")
print(f"Max Sequence Length: {config['max_seq_len']}")
print(f"Dropout Rate: {config['dropout']}")
print(f"")
print(f"Vocabulary Sizes:")
print(f"  Source (Finnish): {len(src_vocab):,}")
print(f"  Target (English): {len(tgt_vocab):,}")
print(f"")
print(f"Training Information:")
print(f"  Epochs Trained: {checkpoint['epoch']}")
print(f"  Final Train Loss: {checkpoint['train_loss']:.4f}")
print(f"  Final Validation Loss: {checkpoint['val_loss']:.4f}")
print(f"  Batch Size: {config['batch_size']}")
print(f"  Learning Rate: {config['learning_rate']}")
print(f"")
print(f"Model Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Device: {device}")
print("=" * 60)